In [38]:
import requests
import boto3
from mypy_boto3_s3 import S3Client
from pprint import pprint
import numpy as np
import json
import tqdm

In [27]:
API_BASE_URL="http://localhost:8000"
BUCKET="cow-detect-maia"



{'model_info': {'cfg_path': 'configs/test/faster_rcnn.yaml',
                'model_metadata': {'backbone_arch': 'resnet50',
                                   'cfg_path': 'configs/test/faster_rcnn.yaml',
                                   'class_name': 'LossWrapper',
                                   'name': 'FasterRCNNResNetFPN',
                                   'num_classes': 7,
                                   'patch_size': 512,
                                   'trainable_backbone_layers': 4,
                                   'weights_path': 'data/models/faster-rcnn/resnet50-100-epochs-tbl4/best_model.pth'},
                'weights_path': 'data/models/faster-rcnn/resnet50-100-epochs-tbl4/best_model.pth'},
 's3_bucket': 'cow-detect-maia'}


In [28]:
# session = boto3.Session(region_name="us-east-1", profile_name="dvc-user")
# s3: S3Client = session.client("s3")
# prefix = f"{region}/2025-11-22/"
# s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix)['Contents'][0]['Key']

In [51]:
TEST_IMAGE_URI = f"s3://{BUCKET}/Time-Measurement/2025-11-22/2af4192ee4de23185abb0e1677b61fd9ea52db61.JPG"

5472 * 3648

19961856

In [39]:
def measure_times(reps: int = 8) -> list[float]:
    app_info = requests.get(f"{API_BASE_URL}/app-info").json()
    model_name = app_info["model_info"]["model_metadata"]["name"]
    model_arch = app_info["model_info"]["model_metadata"].get("backbone_arch", None)

    print(f"{model_name=}   model_arch={model_arch}")

    payload = {"s3_path": TEST_IMAGE_URI}
    times = []
    for _ in tqdm.tqdm(range(reps)):
        resp = requests.post(f"{API_BASE_URL}/predict", json=payload)
        inf_time = resp.json()['inference_time']
        # print(inf_time)
        times.append(inf_time)

    print(f"times = {np.round(np.array(times), 2)!r}")
    print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))
    return times

In [50]:
# FRC-50 cuda : NVIDIA GeForce RTX 2060 (6.0 GB de VRAM)
times = [7.45, 7.11, 7.07, 7.30, 7.34, 7.46, 7.12, 7.28]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

memory = 2.8   # GB

7.27 +/- 0.14


In [40]:
# FRC-50 cpu:

measure_times()
memory = 1500 # MB

model_name='FasterRCNNResNetFPN'   model_arch=resnet50


100%|██████████| 8/8 [05:35<00:00, 41.95s/it]

times = array([40.56, 39.2 , 38.87, 38.1 , 39.75, 42.65, 39.28, 38.79])
39.65 +/- 1.32


[40.56370558300114,
 39.20379866700023,
 38.86731050000162,
 38.09909062500083,
 39.747364832997846,
 42.6478202090002,
 39.28011300000071,
 38.78916891699919]

In [56]:
39.65 / 7.27, 49.98 / 9.32, 43.21 / 4.39, 42.65 / 4.2

(5.453920220082531, 5.3626609442060085, 9.842824601366743, 10.154761904761903)

In [41]:
# FRC-101 cuda:
times = [9.66, 9.20, 9.30, 9.137716260971501, 9.3829946849728, 9.237513903004583, 9.202632041997276, 9.416010013024788]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))
memory = 2.9  # GB

9.32 +/- 0.16


In [42]:
# FRC-101 cpu:
measure_times()

model_name='FasterRCNNResNetFPN'   model_arch=resnet101


100%|██████████| 8/8 [06:45<00:00, 50.66s/it]

times = array([46.82, 46.1 , 47.12, 46.52, 46.03, 46.99, 47.71, 48.54])
46.98 +/- 0.78


[46.819196832999296,
 46.10259291699913,
 47.12076870800229,
 46.515894500000286,
 46.02580591700098,
 46.988853750001,
 47.71303587500006,
 48.535795042000245]

In [44]:
# HN-2-hn cuda:
times = [4.369541400985327, 3.7918004079838283, 3.7768040589871816, 4.021001018001698, 5.97066516103223, 4.028331107983831, 4.044117910030764, 3.852164224954322]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

memory = 2.6 # GB

4.23 +/- 0.68


In [43]:
# HN-2-hn CPU:   Chip:	Apple M3 Pro
measure_times()


model_name='HerdNet'   model_arch=None


100%|██████████| 8/8 [06:17<00:00, 47.20s/it]

times = array([43.69, 43.93, 43.58, 43.21, 43.45, 42.62, 42.41, 42.81])
43.21 +/- 0.51


[43.69013087500207,
 43.928509291999944,
 43.57704808299968,
 43.20568733300024,
 43.45425945899842,
 42.617420917002164,
 42.407587958001386,
 42.809150499997486]

In [45]:
# HN-2 cuda
times = [4.634246086003259, 3.932469096966088, 4.064917451993097, 4.624724559020251, 3.981831144017633, 4.66037645196775, 4.679108470038045, 4.5809408979839645]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

memory = 2.6 # GB

4.39 +/- 0.31


In [48]:
measure_times()


model_name='HerdNet'   model_arch=None


100%|██████████| 8/8 [06:21<00:00, 47.69s/it]

times = array([42.27, 42.84, 42.79, 42.7 , 42.19, 43.16, 43.23, 44.27])
42.93 +/- 0.61


[42.26855470800001,
 42.841075332999026,
 42.79077866700027,
 42.69701079100196,
 42.189605332998326,
 43.158369459000824,
 43.23180824999872,
 44.26691662500161]

In [47]:
# HN-3 cuda
times = [4.883444601029623, 3.918730864999816, 3.9554799090255983, 4.147527578053996, 4.240999165980611, 4.01755037996918, 4.255680132017005, 4.220275408995803b4]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

4.2 +/- 0.28


In [49]:
measure_times()

model_name='HerdNet'   model_arch=None


100%|██████████| 8/8 [06:09<00:00, 46.25s/it]

times = array([42.04, 42.19, 43.02, 43.13, 42.46, 42.53, 42.73, 43.15])
42.65 +/- 0.4


[42.0395374159998,
 42.187332207999134,
 43.01560016700023,
 43.127359291000175,
 42.457790292002755,
 42.5259122919997,
 42.72509524999987,
 43.146184292003454]